In [ ]:
##Script used for obtaining keras results with DNA kmers, they are way worse than with pfam counts.

In [252]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\dictionariesForKmerCounts

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\dictionariesForKmerCounts


In [253]:
import numpy as np

In [254]:
kmerToColumn={}
i=0
numberOfClusters=0
inputFile = open("length4Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        numberOfClusters +=1
        continue
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        if kmer[0] not in kmerToColumn:
            kmerToColumn[kmer[0]]=i
            i+=1
inputFile.close()

In [255]:
inputVectorMatrix= np.zeros((numberOfClusters,i))
i=0
inputFile = open("length4Processed.txt")
for line in inputFile:
    if line [0:3]== "BGC":
        continue
    thisClusterDictionary={}
    line = line.strip("\n")
    line = line.strip(";")
    line = line.split(";")
    for kmer in line:
        kmer=kmer.split(":")
        thisClusterDictionary[kmer[0]] =kmer[1]
    for key in kmerToColumn:
        if key in thisClusterDictionary.keys():
            inputVectorMatrix[i,kmerToColumn[key]] = thisClusterDictionary[key]
    i+=1
inputFile.close()

In [256]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [257]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints


In [258]:
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [259]:
fingerprintVectorList=[]##List of vectors
numberOfClusters=0
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))
    numberOfClusters+=1
yData=fingerprintVectorList
xData=inputVectorMatrix

In [260]:
####THIS CAN BE ACTIVATED FOR NORMALISATION OF EACH COLUMN
#numberOfFeatures= xData.shape[1]
#for i in range (0,numberOfFeatures):
   # mean=xData.mean(0)[i]
   # sd= xData.std(0)[i]
    #for j in range (0, numberOfClusters):
        #xData[j][i]= (xData[j][i]-mean)/sd

In [261]:
yData=np.concatenate(yData,axis=0)
yData=yData.reshape(numberOfClusters,306)
#The reshape puts the data as one row for each training example
inputDimension= xData.shape[1]

In [262]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(numberOfClusters), numberOfClusters)

In [263]:
xTestData = np.asarray([xData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
yTestData = np.asarray([yData[i] for i in listOfRandomNumbers[618:824]],dtype=int)
xTrainingData = np.asarray([xData[i] for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)
yTrainingData = np.asarray([yData[i]for i in (listOfRandomNumbers[:618]+listOfRandomNumbers[824:])],dtype=int)

In [264]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import initializers
nn=[inputDimension, 30, 306]
##The weights of the model are saved and loaded each time the learning rate is changed

In [265]:
lr=0.03
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['binary_crossentropy'])
model.fit(xTrainingData, yTrainingData, epochs=30)
model.save_weights('trial.h5')

Epoch 1/30
825/825 [==============================] - 22s 27ms/step - loss: 0.7535 - binary_crossentropy: 0.7535
Epoch 2/30
825/825 [==============================] - 0s 112us/step - loss: 0.6849 - binary_crossentropy: 0.6849
Epoch 3/30
825/825 [==============================] - 0s 116us/step - loss: 0.6785 - binary_crossentropy: 0.6785
Epoch 4/30
825/825 [==============================] - 0s 117us/step - loss: 0.6691 - binary_crossentropy: 0.6691
Epoch 5/30
825/825 [==============================] - 0s 115us/step - loss: 0.6586 - binary_crossentropy: 0.6586
Epoch 6/30
825/825 [==============================] - 0s 120us/step - loss: 0.6384 - binary_crossentropy: 0.6384
Epoch 7/30
825/825 [==============================] - 0s 120us/step - loss: 0.6108 - binary_crossentropy: 0.6108
Epoch 8/30
825/825 [==============================] - 0s 114us/step - loss: 0.5659 - binary_crossentropy: 0.5659
Epoch 9/30
825/825 [==============================] - 0s 112us/step - loss: 0.4763 - binary_cros

In [266]:
lr=0.01
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['binary_crossentropy'])
model.load_weights("trial.h5")
model.fit(xTrainingData, yTrainingData, epochs=30)
model.save_weights('trial.h5')

Epoch 1/30
825/825 [==============================] - 23s 28ms/step - loss: 0.1671 - binary_crossentropy: 0.1671
Epoch 2/30
825/825 [==============================] - 0s 121us/step - loss: 0.1635 - binary_crossentropy: 0.1635
Epoch 3/30
825/825 [==============================] - 0s 115us/step - loss: 0.1633 - binary_crossentropy: 0.1633
Epoch 4/30
825/825 [==============================] - 0s 114us/step - loss: 0.1638 - binary_crossentropy: 0.1638
Epoch 5/30
825/825 [==============================] - 0s 120us/step - loss: 0.1635 - binary_crossentropy: 0.1635
Epoch 6/30
825/825 [==============================] - 0s 120us/step - loss: 0.1632 - binary_crossentropy: 0.1632
Epoch 7/30
825/825 [==============================] - 0s 118us/step - loss: 0.1629 - binary_crossentropy: 0.1629
Epoch 8/30
825/825 [==============================] - 0s 123us/step - loss: 0.1630 - binary_crossentropy: 0.1630
Epoch 9/30
825/825 [==============================] - 0s 124us/step - loss: 0.1634 - binary_cros

In [267]:
lr=0.003
model=Sequential()
model.add(Dense(nn[0], input_dim=inputDimension, activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1) ))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu', kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[1], activation = 'relu',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.add(Dense(nn[2], activation = 'sigmoid',kernel_initializer=initializers.random_normal(stddev=0.1)))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics = ['binary_crossentropy'])
model.load_weights("trial.h5")
model.fit(xTrainingData, yTrainingData, epochs=30)
model.save_weights('trial.h5')

Epoch 1/30
825/825 [==============================] - 22s 27ms/step - loss: 0.1601 - binary_crossentropy: 0.1601
Epoch 2/30
825/825 [==============================] - 0s 114us/step - loss: 0.1598 - binary_crossentropy: 0.1598
Epoch 3/30
825/825 [==============================] - 0s 121us/step - loss: 0.1599 - binary_crossentropy: 0.1599
Epoch 4/30
825/825 [==============================] - 0s 122us/step - loss: 0.1599 - binary_crossentropy: 0.1599
Epoch 5/30
825/825 [==============================] - 0s 118us/step - loss: 0.1598 - binary_crossentropy: 0.1598
Epoch 6/30
825/825 [==============================] - 0s 124us/step - loss: 0.1597 - binary_crossentropy: 0.1597
Epoch 7/30
825/825 [==============================] - 0s 118us/step - loss: 0.1601 - binary_crossentropy: 0.1601
Epoch 8/30
825/825 [==============================] - 0s 118us/step - loss: 0.1598 - binary_crossentropy: 0.1598
Epoch 9/30
825/825 [==============================] - 0s 120us/step - loss: 0.1600 - binary_cros

In [268]:
def gaussianKernel (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [269]:
predictions = model.predict(xTestData)
for i in range (0, len (predictions)):
    for j in range (0, len (predictions[0])):##Rounds up and down the numbers to 0 and 1
        if predictions[i][j]<0.5:
            predictions[i][j] = 0
        else:
            predictions[i][j] = 1
score = np.zeros((206, 206))##Creates the ouputfile ready to be opened with analysing Scores notebook

for i in range (0, 206):
    for j in range (0, 206):
        score[i,j]= gaussianKernel(predictions[i],yTestData[j])
np.save("kmersKerasScore",score)

112
94
